### Note: 
Code used based heavily on RealSense library repo - see here: https://github.com/IntelRealSense/librealsense/blob/master/wrappers/python/examples/align-depth2color.py


In [13]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os

In [14]:
# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
camera_intrinsics = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
align = rs.align(align_to)

Depth Scale is:  0.0010000000474974513


In [15]:
camera_intrinsics

width: 640, height: 480, ppx: 322.774, ppy: 236.928, fx: 615.113, fy: 615.117, model: 2, coeffs: [0, 0, 0, 0, 0]

In [18]:
frame_num =  1
output_dir = "/home/stanlew/src/kpam/data/test_images"
scene_label = "out_of_category/metal_mug_cardboard_handle"

In [20]:
frames = pipeline.wait_for_frames()
# frames.get_depth_frame() is a 640x360 depth image

# Align the depth frame to color frame
aligned_frames = align.process(frames)

# Get aligned frames
aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
color_frame = aligned_frames.get_color_frame()

# Validate that both frames are valid
if aligned_depth_frame and color_frame:
    
    print("writing images for frame number: {0}".format(frame_num))
    depth_image = np.asanyarray(aligned_depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    
    # Remove background - Set pixels further than clipping_distance to grey
    grey_color = 153
    #depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
    #bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)

    # Render images
    #depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
    if(not os.path.exists(os.path.join(output_dir, scene_label))):
        os.makedirs(os.path.join(output_dir, scene_label))
    cv2.imwrite(os.path.join(output_dir, scene_label, "{0}_rgb.png".format(frame_num)), color_image)
    cv2.imwrite(os.path.join(output_dir, scene_label, "{0}_depth.png".format(frame_num)), depth_image)

    frame_num += 1
else:
    print("either aligned_depth_frame or color_frame incorrect!")

writing images for frame number: 2


In [12]:
pipeline.stop()